In [ ]:
import os 
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import requests
import json
import gradio as gr



In [ ]:
load_dotenv(override=True)
gemini_api_key= os.getenv('GOOGLE_API_KEY')

if gemini_api_key:
    print ("Api key exists ")
else:
    print("API key does not exists")

In [ ]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")


if pushover_user:
    print ("pushover_user exists ")
else:
    print("pushover_user does not exists")

if pushover_token:
    print ("pushover_token exists ")
else:
    print("pushover_token does not exists")


In [ ]:
pushover_url = "https://api.pushover.net/1/messages.json"

def push(message):
    print (f"push : {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message":message}
    requests.post(pushover_url, data=payload)

In [ ]:
push("How are you ashu Ji??")

In [ ]:
def record_user_details(email, name="Not provided", notes="Not provided"):
    push(f"recording interest from {name} with email {email} and notes {notes}")
    return{"recorded": "OK"}


In [ ]:
def record_unknown_questions(question):
    push(f"Recording {question} that is asked which i could not answer!!")
    return {"Recorded": "OK"}

In [ ]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record an user that is interested in connecting and provided email address",
    "parameters": {
        "type": "object",
        "properties" : {
            "email" : {
                "type": "string",
                "description": "The email address of the user"
            },
            "name": {
                "type": "string",
                "description": "The Users name, if they provided it"
            },
            "notes": {
                "type": "string",
                "description": "Additional information about conbversation that worth recording"
            },
            "required": ["email"],
            "additionalProperties": "False"
        }
    }

}

In [ ]:
record_unknown_questions_json = {
    "name": "record_unknown_questions",
    "description": "Always use this tool to record any question that you couldn't be answered as you don't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "question",
                "description": "The question that couldn't be answered"
            },
            "required": ["question"],
            "additionalProperties": "False"
        }
    }
}

In [ ]:
tools = [{"type":"function", "function": record_user_details_json},
         {"type":"function", "function": record_unknown_questions_json}]

In [ ]:
tools

In [ ]:
# Function which will handle tools calls based on if else
def handle_tools_call(tools_calls):
    results= []
    for tool_call in tools_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called {tool_name}", flush=True)

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_questions":
            result = record_unknown_questions(**arguments)
        
        results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id})

In [ ]:
globals()["record_unknown_questions"]("This is really hard question")

In [ ]:
GEMINI_BASE_URL= "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini = OpenAI(
    base_url=GEMINI_BASE_URL,
    api_key=gemini_api_key
)

In [ ]:
reader = PdfReader("../LinkedInProfile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

print("linkedin")

In [ ]:
with open("../summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Ashutosh Jha"

In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [ ]:
def chat(message, history):
    messages = [{"role":"system", "content":system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:
        response = gemini.chat.completions.create( model="gemini-2.0-flash", messages=messages)
        finish_reason =response.choices[0].finish_reason

        if finish_reason == "tool_calls":
            message= response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tools_call(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True

    return response.choices[0].message.content


In [ ]:
gr.ChatInterface(chat, type="messages").launch()